In [26]:
import pyspark
from operator import add
from pyspark import SparkConf
from pyspark.ml.feature import NGram
from pyspark.sql.functions import col,udf
from pyspark.sql import SQLContext
from operator import add
import numpy as np
from pyspark.mllib.linalg.distributed import CoordinateMatrix, MatrixEntry
import string
from pyspark.mllib.util import MLUtils
from pyspark.ml.linalg import Vectors

from pyspark.mllib.linalg import Vectors as MLLibVectors
from pyspark.sql.functions import udf
from pyspark.ml.linalg import VectorUDT as VectorUDTML

from pyspark.ml import Pipeline
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.feature import IndexToString, StringIndexer, VectorIndexer
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

as_ml = udf(lambda v: v.asML() if v is not None else None, VectorUDTML())

#sc = pyspark.SparkContext('local[*]',appName="DocClassification")
#sqlc = SQLContext(sc)



In [38]:
def swap(x):
    return (x[1],x[0])

strs = sc.parallelize(["This is something I really want to test.",
                       "To test something is something I really want to do.",
                       "I need to do something that I want to do which is to test something.",
                       "Training something is something that I needed to do.",
                       "I trained something and that something was what I needed to do.",
                       "Something I needed to do was to train something.",
                       "Something trained is something that needed to be done.",
                       "If you want to train something something has to be trained.",
                       "Train it like you've never trained it before."])
strs = strs.zipWithIndex()\
            .map(lambda x: swap(x))\
            .flatMapValues(lambda x: x.split())\
            .mapValues(lambda x: x.lower().strip(string.punctuation))\
            .map(lambda x: ((x[0],x[1]),1))
            
labs = sc.parallelize([(0,1),(1,0),(2,1),(3,1),(4,0),(5,0),(6,1),]).toDF(["did","label"])
            
print(strs.collect())
labs.show()

sstrs = strs.reduceByKey(add)\
            .map(lambda x: (x[0][1],(x[0][0],x[1])))
    
vocab = sstrs.keys().distinct().zipWithIndex()
print(sstrs.collect())

[((0, 'this'), 1), ((0, 'is'), 1), ((0, 'something'), 1), ((0, 'i'), 1), ((0, 'really'), 1), ((0, 'want'), 1), ((0, 'to'), 1), ((0, 'test'), 1), ((1, 'to'), 1), ((1, 'test'), 1), ((1, 'something'), 1), ((1, 'is'), 1), ((1, 'something'), 1), ((1, 'i'), 1), ((1, 'really'), 1), ((1, 'want'), 1), ((1, 'to'), 1), ((1, 'do'), 1), ((2, 'i'), 1), ((2, 'need'), 1), ((2, 'to'), 1), ((2, 'do'), 1), ((2, 'something'), 1), ((2, 'that'), 1), ((2, 'i'), 1), ((2, 'want'), 1), ((2, 'to'), 1), ((2, 'do'), 1), ((2, 'which'), 1), ((2, 'is'), 1), ((2, 'to'), 1), ((2, 'test'), 1), ((2, 'something'), 1), ((3, 'training'), 1), ((3, 'something'), 1), ((3, 'is'), 1), ((3, 'something'), 1), ((3, 'that'), 1), ((3, 'i'), 1), ((3, 'needed'), 1), ((3, 'to'), 1), ((3, 'do'), 1), ((4, 'i'), 1), ((4, 'trained'), 1), ((4, 'something'), 1), ((4, 'and'), 1), ((4, 'that'), 1), ((4, 'something'), 1), ((4, 'was'), 1), ((4, 'what'), 1), ((4, 'i'), 1), ((4, 'needed'), 1), ((4, 'to'), 1), ((4, 'do'), 1), ((5, 'something'), 1), 

In [103]:
sstrs = strs.reduceByKey(add)\
            .map(lambda x: (x[0][1],(x[0][0],x[1])))
    
vocab = sstrs.keys().distinct().zipWithIndex()

sstrs = sstrs.join(vocab).map(lambda x: (x[1][0][0],x[1][1],float(x[1][0][1])))
print(sstrs.sortBy(lambda x: x[0]).collect())

sstrs = sstrs.map(lambda x: MatrixEntry(x[0],x[1],float(x[2])))
mat = CoordinateMatrix(sstrs).toIndexedRowMatrix().rows.toDF(["did","features"])
print(mat.head())
mat.show()

[(0, 0, 1.0), (0, 2, 1.0), (0, 3, 1.0), (0, 13, 1.0), (0, 5, 1.0), (0, 14, 1.0), (0, 15, 1.0), (0, 18, 1.0), (1, 0, 1.0), (1, 2, 2.0), (1, 3, 1.0), (1, 1, 1.0), (1, 5, 2.0), (1, 14, 1.0), (1, 15, 1.0), (1, 18, 1.0), (2, 0, 2.0), (2, 2, 2.0), (2, 9, 1.0), (2, 10, 1.0), (2, 11, 1.0), (2, 1, 2.0), (2, 5, 3.0), (2, 14, 1.0), (2, 15, 1.0), (2, 18, 1.0), (3, 0, 1.0), (3, 2, 2.0), (3, 9, 1.0), (3, 16, 1.0), (3, 1, 1.0), (3, 5, 1.0), (3, 14, 1.0), (3, 17, 1.0), (4, 0, 2.0), (4, 2, 2.0), (4, 7, 1.0), (4, 9, 1.0), (4, 16, 1.0), (4, 20, 1.0), (4, 1, 1.0), (4, 4, 1.0), (4, 5, 1.0), (4, 8, 1.0), (5, 0, 1.0), (5, 2, 2.0), (5, 6, 1.0), (5, 16, 1.0), (5, 1, 1.0), (5, 4, 1.0), (5, 5, 2.0), (6, 2, 2.0), (6, 7, 1.0), (6, 9, 1.0), (6, 16, 1.0), (6, 19, 1.0), (6, 5, 1.0), (6, 12, 1.0), (6, 14, 1.0)]
Row(did=0, features=SparseVector(21, {0: 1.0, 2: 1.0, 3: 1.0, 5: 1.0, 13: 1.0, 14: 1.0, 15: 1.0, 18: 1.0}))
+---+--------------------+
|did|            features|
+---+--------------------+
|  0|(21,[0,2,3,5,13,

In [108]:
fin = mat.join(labs,['did']).drop("did")
fin.show()


+--------------------+-----+
|            features|label|
+--------------------+-----+
|(21,[0,2,3,5,13,1...|    1|
|(21,[2,5,7,9,12,1...|    1|
|(21,[0,1,2,4,5,6,...|    0|
|(21,[0,1,2,3,5,14...|    0|
|(21,[0,1,2,5,9,14...|    1|
|(21,[0,1,2,5,9,10...|    1|
|(21,[0,1,2,4,5,7,...|    0|
+--------------------+-----+



In [109]:
result = fin.withColumn("features", as_ml("features"))

result.show()

+--------------------+-----+
|            features|label|
+--------------------+-----+
|(21,[0,2,3,5,13,1...|    1|
|(21,[2,5,7,9,12,1...|    1|
|(21,[0,1,2,4,5,6,...|    0|
|(21,[0,1,2,3,5,14...|    0|
|(21,[0,1,2,5,9,14...|    1|
|(21,[0,1,2,5,9,10...|    1|
|(21,[0,1,2,4,5,7,...|    0|
+--------------------+-----+



In [110]:

fin.rdd.collect()


[Row(features=SparseVector(21, {0: 1.0, 2: 1.0, 3: 1.0, 5: 1.0, 13: 1.0, 14: 1.0, 15: 1.0, 18: 1.0}), label=1),
 Row(features=SparseVector(21, {2: 2.0, 5: 1.0, 7: 1.0, 9: 1.0, 12: 1.0, 14: 1.0, 16: 1.0, 19: 1.0}), label=1),
 Row(features=SparseVector(21, {0: 1.0, 1: 1.0, 2: 2.0, 4: 1.0, 5: 2.0, 6: 1.0, 16: 1.0}), label=0),
 Row(features=SparseVector(21, {0: 1.0, 1: 1.0, 2: 2.0, 3: 1.0, 5: 2.0, 14: 1.0, 15: 1.0, 18: 1.0}), label=0),
 Row(features=SparseVector(21, {0: 1.0, 1: 1.0, 2: 2.0, 5: 1.0, 9: 1.0, 14: 1.0, 16: 1.0, 17: 1.0}), label=1),
 Row(features=SparseVector(21, {0: 2.0, 1: 2.0, 2: 2.0, 5: 3.0, 9: 1.0, 10: 1.0, 11: 1.0, 14: 1.0, 15: 1.0, 18: 1.0}), label=1),
 Row(features=SparseVector(21, {0: 2.0, 1: 1.0, 2: 2.0, 4: 1.0, 5: 1.0, 7: 1.0, 8: 1.0, 9: 1.0, 16: 1.0, 20: 1.0}), label=0)]

In [111]:
labelIndexer1 = StringIndexer(inputCol="label", outputCol="indexedLabel").fit(result)
featureIndexer1 =\
    VectorIndexer(inputCol="features", outputCol="indexedFeatures", maxCategories=4).fit(result)
(trainingData, testData) = result.randomSplit([0.7, 0.3])
rf = RandomForestClassifier(labelCol="indexedLabel", featuresCol="indexedFeatures", numTrees=10)
labelConverter = IndexToString(inputCol="prediction", outputCol="predictedLabel",
                               labels=labelIndexer.labels)
pipeline = Pipeline(stages=[labelIndexer, featureIndexer, rf, labelConverter])
model = pipeline.fit(trainingData)

Py4JJavaError: An error occurred while calling o5006.fit.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 20 in stage 2169.0 failed 1 times, most recent failure: Lost task 20.0 in stage 2169.0 (TID 11955, localhost, executor driver): org.apache.spark.SparkException: Failed to execute user defined function($anonfun$5: (string) => double)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIterator.processNext(Unknown Source)
	at org.apache.spark.sql.execution.BufferedRowIterator.hasNext(BufferedRowIterator.java:43)
	at org.apache.spark.sql.execution.WholeStageCodegenExec$$anonfun$8$$anon$1.hasNext(WholeStageCodegenExec.scala:395)
	at scala.collection.Iterator$$anon$11.hasNext(Iterator.scala:408)
	at scala.collection.Iterator$$anon$11.hasNext(Iterator.scala:408)
	at scala.collection.Iterator$$anon$11.hasNext(Iterator.scala:408)
	at scala.collection.Iterator$$anon$11.hasNext(Iterator.scala:408)
	at scala.collection.Iterator$$anon$10.hasNext(Iter

[Row(did=0, features=SparseVector(12, {0: 1.0, 2: 1.0, 3: 1.0, 4: 1.0, 8: 1.0, 9: 1.0, 10: 1.0, 11: 1.0}), label=1),
 Row(did=1, features=SparseVector(12, {0: 1.0, 1: 1.0, 2: 2.0, 3: 1.0, 4: 2.0, 9: 1.0, 10: 1.0, 11: 1.0}), label=0),
 Row(did=2, features=SparseVector(12, {0: 2.0, 1: 2.0, 2: 2.0, 4: 3.0, 5: 1.0, 6: 1.0, 7: 1.0, 9: 1.0, 10: 1.0, 11: 1.0}), label=1)]

In [51]:
data = sqlc.read.format("libsvm").load("data/sample_libsvm_data.txt")
data.show()
labelIndexer = StringIndexer(inputCol="label", outputCol="indexedLabel").fit(data)
featureIndexer =\
    VectorIndexer(inputCol="features", outputCol="indexedFeatures", maxCategories=4).fit(data)
data.show()


+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|(692,[127,128,129...|
|  1.0|(692,[158,159,160...|
|  1.0|(692,[124,125,126...|
|  1.0|(692,[152,153,154...|
|  1.0|(692,[151,152,153...|
|  0.0|(692,[129,130,131...|
|  1.0|(692,[158,159,160...|
|  1.0|(692,[99,100,101,...|
|  0.0|(692,[154,155,156...|
|  0.0|(692,[127,128,129...|
|  1.0|(692,[154,155,156...|
|  0.0|(692,[153,154,155...|
|  0.0|(692,[151,152,153...|
|  1.0|(692,[129,130,131...|
|  0.0|(692,[154,155,156...|
|  1.0|(692,[150,151,152...|
|  0.0|(692,[124,125,126...|
|  0.0|(692,[152,153,154...|
|  1.0|(692,[97,98,99,12...|
|  1.0|(692,[124,125,126...|
+-----+--------------------+
only showing top 20 rows

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|(692,[127,128,129...|
|  1.0|(692,[158,159,160...|
|  1.0|(692,[124,125,126...|
|  1.0|(692,[152,153,154...|
|  1.0|(692,[151,152,153...|
|  0.0|(692,[129,130,131...|
|  1.0|(692,[158,

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|(692,[127,128,129...|
|  1.0|(692,[158,159,160...|
|  1.0|(692,[124,125,126...|
|  1.0|(692,[152,153,154...|
|  1.0|(692,[151,152,153...|
|  0.0|(692,[129,130,131...|
|  1.0|(692,[158,159,160...|
|  1.0|(692,[99,100,101,...|
|  0.0|(692,[154,155,156...|
|  0.0|(692,[127,128,129...|
|  1.0|(692,[154,155,156...|
|  0.0|(692,[153,154,155...|
|  0.0|(692,[151,152,153...|
|  1.0|(692,[129,130,131...|
|  0.0|(692,[154,155,156...|
|  1.0|(692,[150,151,152...|
|  0.0|(692,[124,125,126...|
|  0.0|(692,[152,153,154...|
|  1.0|(692,[97,98,99,12...|
|  1.0|(692,[124,125,126...|
+-----+--------------------+
only showing top 20 rows



In [12]:
def fun1(accum,n):
    if len(n)==0 : return accum
    else :return accum + "gs://uga-dsp/project2/data/bytes/"+n+".asm,"
files = sc.textFile("data/X_small_train.txt")
wtf = files.reduce(fun1)
print(wtf[:-6])

DvdM5Zpx96qKuN3cAt1ygs://uga-dsp/project2/data/bytes/5QpgRV2cqU9wvjBist1a.asm,gs://uga-dsp/project2/data/bytes/2F6ZfVCQRi3vrwcj4zxL.asm,gs://uga-dsp/project2/data/bytes/1KjZ4An78sOytkzgRL0E.asm,gs://uga-dsp/project2/data/bytes/JnHGRI2v5NuB9lpsEOCS.asm,gs://uga-dsp/project2/data/bytes/fvTz3rSPVQJGelO49Rn0.asm,gs://uga-dsp/project2/data/bytes/Akbqr8c0E31QoOTwKefL.asm,gs://uga-dsp/project2/data/bytes/7wFaWUhcDgonLtQdqpy2.asm,gs://uga-dsp/project2/data/bytes/C1IklXQF4DJiB8stvR39.asm,gs://uga-dsp/project2/data/bytes/fOoBREwIcNmD8u6Kts75.asm,gs://uga-dsp/project2/data/bytes/8jrxyaQiGJb5qcsOmMAv.asm,gs://uga-dsp/project2/data/bytes/CQwNADZxFXBh9fUlOuao.asm,gs://uga-dsp/project2/data/bytes/FEA6d5CzUyqoZPS8tfQk.asm,gs://uga-dsp/project2/data/bytes/4xPwVZd8CpJEAUIebmvB.asm,gs://uga-dsp/project2/data/bytes/BmMDeyRaJzQ39WPK2VIs.asm,gs://uga-dsp/project2/data/bytes/fzgZyTMiq8lmuKkhdS27.asm,gs://uga-dsp/project2/data/bytes/EzxVk2DCYKWZpng39AoS.asm,gs://uga-dsp/project2/data/bytes/e5fn4GutJO601CgUSZF

In [34]:
vocab = sstrs.keys()
N = strs.map(lambda x: x[0][0]).distinct().count()
print(strs.keys().map(lambda x: x[0]).distinct().count())


9


In [35]:
strs = strs.map(lambda x: (x,np.zeros(N)))
strs.collect()

[(((0, 'this'), 1), array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.])),
 (((0, 'is'), 1), array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.])),
 (((0, 'something'), 1), array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.])),
 (((0, 'i'), 1), array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.])),
 (((0, 'really'), 1), array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.])),
 (((0, 'want'), 1), array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.])),
 (((0, 'to'), 1), array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.])),
 (((0, 'test'), 1), array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.])),
 (((1, 'to'), 1), array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.])),
 (((1, 'test'), 1), array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.])),
 (((1, 'something'), 1), array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.])),
 (((1, 'is'), 1), array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.])),
 (((1, 'something'), 1), array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.])),
 (((1, 'i'), 1), ar